In [32]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
# Read the data
ames = pd.read_csv("/Users/benji/Documents/Machine Learning/Data/AmesHousing.csv")

# Get rid of columns with mostly NaN values
good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

# Drop other NAs
ames = ames.dropna()

In [4]:
ames

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,7937,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,8885,Pave,IR1,Low,AllPub,Inside,...,0,0,0,0,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,10441,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,10010,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,4,2006,WD,Normal,170000


In [27]:
X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]


ct = ColumnTransformer(
  [
    ("dummify", 
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize", 
    StandardScaler(), 
    make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

lr_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("ridgeRegression", Ridge(alpha= 1))]
)

In [28]:
ridgeModel = lr_pipeline_1.fit(X,y)
pred = ridgeModel.predict(X)

mse = mean_squared_error(y,pred)
r2 = r2_score(y,pred)
print(r2)
print(mse)

0.9157036706307324
535795379.517413


In [30]:
alphas =[0.001, 0.01, 0.1, 1, 10]

for alpha in alphas:
    lr_pipeline_2 = Pipeline(
  [("preprocessing", ct),
  ("ridgeRegression", Ridge(alpha= alpha))]
)
    tuneModel = lr_pipeline_2.fit(X,y)
    pred = tuneModel.predict(X)

    mse = mean_squared_error(y,pred)
    r2 = r2_score(y,pred)
    print(f" R-square value: {r2}, MSE: {mse}")
    

 R-square value: 0.9211143674272932, MSE: 501404482.9593017
 R-square value: 0.9211113302114817, MSE: 501423787.78800076
 R-square value: 0.9208744732344534, MSE: 502929272.95974874
 R-square value: 0.9157036706307324, MSE: 535795379.517413
 R-square value: 0.9040209838462121, MSE: 610051632.978637


In [44]:
lasso_pipeline = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("lasso", Lasso())
    ]
)

# Parameter grid for alpha tuning
param_grid = {
    "lasso__alpha": alphas
}

# Perform grid search cross-validation
lasso_search = GridSearchCV(lasso_pipeline, param_grid, cv=5, scoring="neg_mean_squared_error")
lasso_search.fit(X, y)  # Replace X, y with your full Ames data features and target

# Best LASSO model and alpha
best_lasso_model = lasso_search.best_estimator_
best_alpha_lasso = lasso_search.best_params_["lasso__alpha"]

print(f"Best LASSO alpha: {best_alpha_lasso}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.109e+11, tolerance: 1.348e+09
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.466e+11, tolerance: 1.474e+09
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Best LASSO alpha: 10


In [35]:
# Set up and fit the Ridge model (tuned similarly to LASSO)
ridge_pipeline = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("ridge", Ridge(alpha=best_alpha_lasso))  # Use the same alpha as a comparison point
    ]
)
ridge_pipeline.fit(X, y)

# Set up and fit the OLS model
ols_pipeline = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("ols", LinearRegression())
    ]
)
ols_pipeline.fit(X, y)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummify',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x30284ae90>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x302a52450>)])),
                ('ols', LinearRegression())])

In [37]:
# Get coefficients for each model
lasso_coefs = best_lasso_model.named_steps["lasso"].coef_
ridge_coefs = ridge_pipeline.named_steps["ridge"].coef_
ols_coefs = ols_pipeline.named_steps["ols"].coef_

# Print or analyze the differences in coefficients
print("LASSO Coefficients:", lasso_coefs)



LASSO Coefficients: [-4.51898958e+03  0.00000000e+00 -0.00000000e+00  4.59256134e+03
  2.31827307e+03 -2.08235173e+03 -1.70965918e+04  8.51299281e-10
 -1.16351324e+03  4.37136675e+03 -0.00000000e+00  0.00000000e+00
 -9.49462247e+03  5.42684544e+03 -9.44561591e+03  1.35861380e+02
  3.64335710e+03 -0.00000000e+00 -0.00000000e+00  7.90332217e+01
  6.36350586e+03 -5.95620348e+03 -2.90507491e+03  0.00000000e+00
 -0.00000000e+00  6.16612008e+03 -1.10426840e+04  4.81758840e+03
  8.94742519e+03  1.31928184e+04  3.75089101e+03 -0.00000000e+00
 -2.66427635e+03  1.46845352e+04 -1.01570542e+04 -3.79902131e+03
  1.02574836e+04  1.09304458e+05 -2.35734919e+03  0.00000000e+00
  7.12699605e+03 -6.36780861e+03 -6.32679579e+03  1.49640904e+04
 -9.25353657e+03  3.49564239e+04  2.80292835e+04 -6.33356337e+03
 -3.72792988e+03 -1.84028112e+03 -3.36693040e+03  1.87963345e+04
  4.46064027e+04 -3.30087424e+02 -2.60186271e+03 -1.33724270e+03
 -0.00000000e+00  8.09252946e+03  6.31479167e+03  1.29896232e+04
 -3.9

In [39]:
print("Ridge Coefficients:", ridge_coefs)

Ridge Coefficients: [-4.90649460e+03  2.34694314e+03 -1.22917740e+03  4.62091189e+03
  2.20428500e+03 -3.03646802e+03 -6.17531422e+03  6.17531422e+03
  2.12575125e+03  7.53147205e+03 -1.22865537e+04  2.62933036e+03
 -1.10363100e+04  9.09887001e+03 -1.49378054e+03  3.43122056e+03
  2.93687893e+03 -2.42162664e+03 -5.15252289e+02  1.43157683e+02
  7.56730650e+03 -5.62761832e+03 -2.75896133e+03  6.76115464e+02
 -3.78227675e+02  5.60816320e+03 -5.22993552e+03 -3.76824510e+03
  2.45905383e+03  5.70332014e+03 -6.25300540e+02 -4.25100049e+03
 -8.51682634e+03  9.07661139e+03 -1.55819272e+04 -1.11163614e+04
  1.72302347e+03  1.23832549e+04 -6.48871508e+03 -3.70052362e+02
  1.59468710e+03 -1.01764141e+04 -1.10938115e+04  4.18711022e+03
 -1.32553889e+04  2.75168983e+04  2.20925274e+04 -9.78994038e+03
 -7.01890866e+03 -6.59706874e+03 -8.97446647e+03  9.31639420e+03
  3.08242659e+04 -5.50691247e+03 -3.74580712e+03 -2.09024782e+03
 -4.15882068e+03  5.95150066e+03  7.86198989e+03  5.63755552e+03
 -5.7

In [40]:
print("OLS Coefficients:", ols_coefs)

OLS Coefficients: [-3.91236093e+16 -3.91236093e+16 -3.91236093e+16 -3.91236093e+16
 -3.91236093e+16 -3.91236093e+16 -4.37772703e+16 -4.37772703e+16
 -1.79569020e+17 -1.79569020e+17 -1.79569020e+17 -1.79569020e+17
 -2.87981219e+17 -2.87981219e+17 -2.87981219e+17 -2.87981219e+17
  7.37997107e+16  7.37997107e+16  7.37997107e+16  1.04522538e+17
  1.04522538e+17  1.04522538e+17  1.04522538e+17  1.04522538e+17
 -1.51648408e+17 -1.51648408e+17 -1.51648408e+17  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  5.03520735e+16
  5.03520735e+16  5.03520735e+16  5.03520735e+16  3.08857780e+16
  3.08857780e+16  3.08857780e+16  3.08857780e+16  3.08857780e+16
  3.088

In [41]:
l1_ratios = [0.2, 0.5, 0.8]
elastic_net_pipeline = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("elasticNet", ElasticNet(max_iter=10000))
    ]
)

# Parameter grid for GridSearchCV
param_grid = {
    "elasticNet__alpha": alphas,
    "elasticNet__l1_ratio": l1_ratios
}

# Perform grid search cross-validation
elastic_net_search = GridSearchCV(elastic_net_pipeline, param_grid, cv=5, scoring="neg_mean_squared_error")
elastic_net_search.fit(X, y)  # Replace X, y with your full Ames data features and target

# Get the best Elastic Net model and parameters
best_elastic_net_model = elastic_net_search.best_estimator_
best_alpha_elastic = elastic_net_search.best_params_["elasticNet__alpha"]
best_l1_ratio_elastic = elastic_net_search.best_params_["elasticNet__l1_ratio"]

print(f"Best Elastic Net alpha: {best_alpha_elastic}")
print(f"Best Elastic Net l1_ratio: {best_l1_ratio_elastic}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.831e+09, tolerance: 1.348e+09
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.136e+09, tolerance: 1.474e+09
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Best Elastic Net alpha: 0.01
Best Elastic Net l1_ratio: 0.5


In [42]:
elastic_net_coefs = best_elastic_net_model.named_steps["elasticNet"].coef_
print("Elastic Net Coefficients:", elastic_net_coefs)

Elastic Net Coefficients: [-4.36707308e+03  2.58930223e+03 -9.19417256e+02  3.85391901e+03
  2.21693900e+03 -3.37366992e+03 -5.42673008e+03  5.42673008e+03
  1.86723281e+03  6.98591103e+03 -1.10890052e+04  2.23386166e+03
 -1.09990709e+04  8.99233086e+03 -1.25763969e+03  3.26437971e+03
  2.26595132e+03 -1.82550582e+03 -4.39445496e+02 -1.94736059e+01
  7.32006917e+03 -5.43410236e+03 -2.44365648e+03  5.78163328e+02
 -6.16362158e+02  5.18209095e+03 -4.56472885e+03 -3.87569980e+03
  1.90088632e+03  4.70492376e+03  3.00238868e+02 -3.50912489e+03
 -8.19981092e+03  9.22438783e+03 -1.44898566e+04 -1.06699459e+04
  1.14082143e+03  9.02194558e+03 -5.36482446e+03 -2.84779724e+02
  1.44588543e+03 -9.23421598e+03 -1.01670811e+04  3.40781950e+03
 -1.23392128e+04  2.60565210e+04  2.09499477e+04 -8.52307135e+03
 -5.88863680e+03 -5.64447612e+03 -8.43567267e+03  8.49411808e+03
  2.82637497e+04 -5.25961559e+03 -3.02122055e+03 -2.05208353e+03
 -4.12029951e+03  5.88166068e+03  7.06090265e+03  4.69101556e+03